In [9]:
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",   
    password="",   
    database="quiz_projectdb1"
)

cursor = db.cursor()


def register_user():
    username = input("Enter username: ")
    password = input("Enter password: ")
    role = input("Enter role (user/admin): ").lower()
    cursor.execute("INSERT INTO users (username, password, role) VALUES (%s, %s, %s)", (username, password, role))
    db.commit()
    print("User registered successfully.")


def login_user():
    username = input("Enter username: ")
    password = input("Enter password: ")
    cursor.execute("SELECT user_id, role FROM users WHERE username = %s AND password = %s", (username, password))
    user = cursor.fetchone()
    
    if user:
        print(f"Login successful. Role: {user[1]}")
        return user[0], user[1]  
    else:
        print("Invalid credentials.")
        return None, None


def admin_create_quiz():
    title = input("Enter quiz title: ")
    cursor.execute("INSERT INTO quizzes (title) VALUES (%s)", (title,))
    db.commit()
    quiz_id = cursor.lastrowid
    print("Quiz created successfully.")
    add_questions_to_quiz(quiz_id)

def add_questions_to_quiz(quiz_id):
    while True:
        question_text = input("Enter question text (or 'quit' to finish): ")
        if question_text.lower() == 'quit':
            break
        correct_answer = input("Enter correct answer (A, B, C, or D): ").upper()
        cursor.execute("INSERT INTO questions (quiz_id, question_text, correct_answer) VALUES (%s, %s, %s)",
                       (quiz_id, question_text, correct_answer))
        db.commit()
        question_id = cursor.lastrowid
        for option_letter in ["A", "B", "C", "D"]:
            option_text = input(f"Enter option {option_letter}: ")
            cursor.execute("INSERT INTO options (question_id, option_text, option_letter) VALUES (%s, %s, %s)",
                           (question_id, option_text, option_letter))
            db.commit()


def take_quiz(user_id):
    cursor.execute("SELECT quiz_id, title FROM quizzes")
    quizzes = cursor.fetchall()
    for quiz in quizzes:
        print(f"{quiz[0]}. {quiz[1]}")
    quiz_id = int(input("Select a quiz by ID: "))

    cursor.execute("SELECT question_id, question_text FROM questions WHERE quiz_id = %s", (quiz_id,))
    questions = cursor.fetchall()
    score = 0

    for question in questions:
        question_id, question_text = question
        print(question_text)
        cursor.execute("SELECT option_letter, option_text FROM options WHERE question_id = %s", (question_id,))
        options = cursor.fetchall()
        for option in options:
            print(f"{option[0]}. {option[1]}")

        user_answer = input("Your answer (A, B, C, or D): ").upper()
        cursor.execute("SELECT correct_answer FROM questions WHERE question_id = %s", (question_id,))
        correct_answer = cursor.fetchone()[0]

        if user_answer == correct_answer:
            score += 1

    cursor.execute("INSERT INTO results (user_id, quiz_id, score) VALUES (%s, %s, %s)", (user_id, quiz_id, score))
    db.commit()
    print(f"Quiz completed. Your score: {score}/{len(questions)}")


def view_results(user_id):
    cursor.execute("""
        SELECT quizzes.title, results.score, results.date_taken
        FROM results
        JOIN quizzes ON results.quiz_id = quizzes.quiz_id
        WHERE results.user_id = %s
    """, (user_id,))
    results = cursor.fetchall()
    for result in results:
        print(f"Quiz: {result[0]}, Score: {result[1]}, Date Taken: {result[2]}")

def admin_menu():
    while True:
        admin_choice = input("1. Create Quiz\n2. Log Out\nSelect an option: ")
        if admin_choice == '1':
            admin_create_quiz()
        elif admin_choice == '2':
            break
        else:
            print("Invalid option.")


def user_menu(user_id):
    while True:
        user_choice = input("1. Take Quiz\n2. View Results\n3. Log Out\nSelect an option: ")
        if user_choice == '1':
            take_quiz(user_id)
        elif user_choice == '2':
            view_results(user_id)
        elif user_choice == '3':
            break
        else:
            print("Invalid option.")


def main():
    print("Welcome to the Quiz System")
    while True:
        choice = input("1. Register\n2. Login\n3. Exit\nSelect an option: ")
        if choice == '1':
            register_user()
        elif choice == '2':
            user_id, role = login_user()
            if user_id:
                if role == 'admin':
                    print("Admin login successful.")
                    admin_menu()
                else:
                    print("User login successful.")
                    user_menu(user_id)
        elif choice == '3':
            print("Exiting the Quiz System.")
            break
        else:
            print("Invalid option.")

if __name__ == "__main__":
    main()



Welcome to the Quiz System


1. Register
2. Login
3. Exit
Select an option:  2
Enter username:  rushi
Enter password:  4123


Login successful. Role: user
User login successful.


1. Take Quiz
2. View Results
3. Log Out
Select an option:  1


1. what is your fav cricket player?


...

Quiz: what is your fav cricket player?, Score: 0, Date Taken: 2024-11-18 11:51:03


1. Take Quiz
2. View Results
3. Log Out
Select an option:  2


Quiz: what is your fav cricket player?, Score: 0, Date Taken: 2024-11-18 11:51:03


1. Take Quiz
2. View Results
3. Log Out
Select an option:  3
1. Register
2. Login
3. Exit
Select an option:  3


Exiting the Quiz System.
